In [1]:
import ee
import geemap
from geemap import ee_initialize
import os
ee_initialize()
#
#https://developers.google.com/earth-engine/tutorials/tutorial_api_07

In [3]:
stations = [
  ee.Feature(
      ee.Geometry.Point(10.5798451965, 46.6862577024), {'name': 'B2 (1490m)'}),
  ee.Feature(
      ee.Geometry.Point(10.5918853513, 46.6917328627), {'name': 'B3 (1930m)'}),
  ee.Feature(
      ee.Geometry.Point(10.5850561988, 46.684249288), {'name': 'P2 (1549m)'})
]
allStations = ee.FeatureCollection(stations)

def do_buffer(t):
    return(t.buffer(35).bounds())
buffer = allStations.map(do_buffer)


In [ ]:
#ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI').clip(buffer)
#ndviParams = {'min': -1, 'max': 1, 'palette': ['#856d08', 'white', 'green']}

In [4]:
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2017-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterBounds(buffer) 

In [5]:
roi = ee.Feature(ee.Geometry.Point(10.5798451965, 46.6862577024), {'name': 'B2 1490m'})
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi).clip(buffer)#image.addBands(ndvi)
withNDVI = s2.map(addNDVI).select("NDVI").toBands()

In [14]:
ndvi_vis = {
  'min': -1.0,'max': 1,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', 
  '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', 
  '023B01','012E01', '011D01', '011301']
}

Map = geemap.Map(center=[ 46.6888079847906,10.586568754186793], zoom = 15)
Map.add_basemap("TERRAIN")
Map.add_basemap("SATELLITE")
Map.addLayer(withNDVI,name="NDVI_series")
Map.addLayer(withNDVI.select(4),ndvi_vis,name="NDVI_image")
Map.addLayer(allStations,name="Monitoring Stations")
Map.set_plot_options(plot_type='bar',add_marker_cluster=False, marker=None)
Map.roi_reducer = ee.Reducer.mean()
Map

Map(center=[46.6888079847906, 10.586568754186793], controls=(WidgetControl(options=['position'], widget=HBox(c…

In [9]:
Map.set_plot_options(add_marker_cluster=True, marker=None)
Map.roi_reducer = ee.Reducer.mean()

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_csv = os.path.join(out_dir, 'points.csv')
out_shp = os.path.join(out_dir, 'ndvi_matsch.shp')
Map.extract_values_to_points(out_shp)

In [ ]:
out_stats = os.path.join(out_dir, 'stats_match.kmz')  

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(withNDVI, buffer, out_stats, statistics_type='MEAN', scale=10)